In [ ]:
import json
import numpy as np
from keras.models import Model
from keras.layers import LSTM, Dense, Dropout, Input, Concatenate
from keras.regularizers import l2

# load the data from the JSON file
with open('/content/sample_data/realistic_trajectory_data-28.json', 'r') as file:
    data = json.load(file)

# extract the trajectories and initial accelerations from the data
trajectories = []
initial_accs = []
for point in data:
    trajectories.append([point['x'], point['y'], point['z']])
    initial_accs.append([point['ax'], point['ay'], point['az']])

# define the number of timesteps and features
num_timesteps = 14 # Number of previous trajectory points to consider
num_pos_features = 3  # x, y, z coordinates

# prepare the training data
X_train_pos = []
X_train_acc = []
y_train = []
for i in range(len(trajectories) - num_timesteps):
    X_train_pos.append(trajectories[i:i+num_timesteps])
    X_train_acc.append(initial_accs[i])
    y_train.append(trajectories[i + num_timesteps])

# convert to numpy arrays for training
X_train_pos = np.array(X_train_pos)
X_train_acc = np.array(X_train_acc)
y_train = np.array(y_train)
input_pos = Input(shape=(num_timesteps, num_pos_features))
input_acc = Input(shape=(3,))

x = LSTM(128, input_shape=(num_timesteps, num_pos_features), return_sequences=True, kernel_regularizer=l2(0.01))(input_pos)
x = Dropout(0.2)(x)
x = LSTM(128, return_sequences=True, kernel_regularizer=l2(0.01))(x)
x = Dropout(0.2)(x)
x = LSTM(64, kernel_regularizer=l2(0.01))(x)
combined = Concatenate(axis=-1)([x, input_acc])
output = Dense(num_pos_features, kernel_regularizer=l2(0.01))(combined)

model = Model(inputs=[input_pos, input_acc], outputs=output)
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit([X_train_pos, X_train_acc], y_train, epochs=200, batch_size=64, validation_split=0.2)
model.save('trajectory_model.h5')

Epoch 1/200
19/19 [==============================] - 9s 180ms/step - loss: 83369.7422 - val_loss: 60998.1797
Epoch 2/200
19/19 [==============================] - 2s 93ms/step - loss: 61885.6641 - val_loss: 45136.9648
Epoch 3/200
19/19 [==============================] - 2s 82ms/step - loss: 46429.5703 - val_loss: 33788.1484
Epoch 4/200
19/19 [==============================] - 2s 84ms/step - loss: 35068.5391 - val_loss: 25345.2754
Epoch 5/200
19/19 [==============================] - 2s 84ms/step - loss: 26512.1152 - val_loss: 18975.9355
Epoch 6/200
19/19 [==============================] - 2s 85ms/step - loss: 20070.4668 - val_loss: 14148.4648
Epoch 7/200
19/19 [==============================] - 2s 84ms/step - loss: 15179.1768 - val_loss: 10562.7256
Epoch 8/200
19/19 [==============================] - 2s 116ms/step - loss: 11531.7139 - val_loss: 7957.8354
Epoch 9/200
19/19 [==============================] - 2s 111ms/step - loss: 8849.8896 - val_loss: 6041.1904
Epoch 10/200
19/19 [========

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
import numpy as np
from keras.models import load_model

# load the trained model
model = load_model('trajectory_model.h5')
num_timesteps = 14  # Number of previous trajectory points to consider
num_pos_features = 3  # x, y, z coordinates

# test the model with a random initial position and acceleration
initial_pos = np.array([1.253435966358851, 0.4282433263299045, 1.457091728002236])
initial_acc = np.array([1142.8386752095407, 390.4571504772658, 1348.4012814138034])

input_seq = [initial_pos] * num_timesteps  # Repeat the initial position

predicted_trajectory = []

for _ in range(8):
    X_test_pos = np.array(input_seq[-num_timesteps:]).reshape(1, num_timesteps, num_pos_features)
    X_test_acc = np.array(initial_acc).reshape(1, 3)
    next_pos = model.predict([X_test_pos, X_test_acc])
    input_seq.append(next_pos[0])
    predicted_trajectory.append(next_pos[0])

print(initial_pos)
for pos in predicted_trajectory:
    print(pos)


1/1 [==============================] - 0s 31ms/step
[1.25343597 0.42824333 1.45709173]
[24.158562  15.970624   5.9580307]
[23.510908  15.332196   5.6641626]
[22.38113   14.379291   5.2773895]
[21.212128  13.48346    4.9568768]
[20.022474 12.656373  4.686505]
[18.72488  11.819104  4.434548]
[17.26544  10.940204  4.181557]
[16.014122  10.319083   3.9131742]
